## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-09-30-00 +0000,wsj,How Hedge Funds Won Big on an Obscure Drugmaker,https://www.wsj.com/finance/investing/abivax-h...
1,2025-08-30-09-30-00 +0000,wsj,"Six Months Into Tariffs, Businesses Have No Id...",https://www.wsj.com/business/retail/trump-tari...
2,2025-08-30-09-10-42 +0000,wapo,"Shoppers met with canceled orders, delays as d...",https://www.washingtonpost.com/business/2025/0...
3,2025-08-30-09-10-30 +0000,bbc,"Russia launches 'massive' attack on Ukraine, a...",https://www.bbc.com/news/articles/c4g614x9kqko...
4,2025-08-30-09-01-57 +0000,nyt,A “Fall of Rome” Candle by JD and Kate Industr...,https://www.nytimes.com/2025/08/30/style/fall-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2446/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,60
152,court,18
73,new,14
221,up,11
66,president,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...,186
90,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,137
116,2025-08-29-22-22-47 +0000,cbc,U.S. appeals court finds Trump tariffs unlawfu...,https://www.cbc.ca/news/world/appeals-court-tr...,132
119,2025-08-29-22-04-53 +0000,nypost,Trump’s tariffs ruled illegal in stunning blow...,https://nypost.com/2025/08/29/us-news/trumps-t...,132
177,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...,125


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,186,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...
64,47,2025-08-30-01-42-35 +0000,nypost,Anti-Israel vandals target ritzy NYC building ...,https://nypost.com/2025/08/29/us-news/anti-isr...
177,47,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...
257,45,2025-08-29-14-03-22 +0000,nypost,California Gov. Newsom says crime is higher in...,https://nypost.com/2025/08/29/us-news/californ...
72,43,2025-08-30-01-18-37 +0000,nypost,Minneapolis Catholic school shooter Robin West...,https://nypost.com/2025/08/29/us-news/pile-of-...
90,42,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
263,39,2025-08-29-13-25-00 +0000,wsj,Trump revokes former Vice President Kamala Har...,https://www.wsj.com/politics/policy/trump-revo...
31,33,2025-08-30-06-52-08 +0000,nypost,Karen Read tells prosecutors ‘you lost big tim...,https://nypost.com/2025/08/30/us-news/karen-re...
107,33,2025-08-29-23-12-00 +0000,wsj,A group of House lawmakers from across the pol...,https://www.wsj.com/politics/policy/bipartisan...
111,32,2025-08-29-22-49-38 +0000,nypost,Top Zelensky adviser invites Trump special env...,https://nypost.com/2025/08/29/world-news/top-z...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
